# Basic DataFrame manipulation in `pandas`

## Introduction

Today we will learn how to manipulate tables in Python using the **`pandas`** library.

`pandas` stands for "Python Data Analysis". It is a Python library that facilitates a wide range of data analysis and manipulation. Before, you saw basic data structures in Python such as lists and dictionaries. While you can build a basic data table using nested lists in Python (similar to an Excel spreadsheet), they get quite difficult to work with. By contrast, in `pandas` the table data structure, known as a **`DataFrame`**, is a first-class citizen and you can easily manipulate your data thinking of it in rows and columns.

## DataFrames and Series

If you've ever used or heard of R or SQL before, `pandas` brings some functionality from each of these to Python, allowing you to structure and filter data more efficiently than when using pure Python. This efficiency is seen in two distinct ways:

* Code written using `pandas` will often run faster than scripts written in pure Python
* Code written using `pandas` will often contain far fewer lines of code than the equivalent code written in pure Python

At the core of the ```pandas``` library are two fundamental data structures/objects:
1. [Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html)
2. [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html)

A **```Series```** object stores single-column data along with an **index**. An index is just a way of "numbering" the ```Series``` object.

A **```DataFrame```** object is a two-dimensional tabular data structure with labeled axes. It is conceptually helpful to think of a DataFrame object as a collection of Series objects. Namely, think of each column in a DataFrame as a single Series object, where each of these Series objects shares a common index -  the index of the DataFrame object.

The following diagrams show the differences between a Series and a DataFrame. This is a DataFrame - notice that it has an index for its columns and an index for its rows:

<img src="data/images/dataframe.jpg" alt="A Dataframe" style="width: 500px;"/>

If you *slice* a DataFrame to extract just **one** column, you'll get a Series. Series can also be created from scratch, independent of any DataFrame. This is a Series:

<img src="data/images/series.jpg" alt="A Series" style="width: 600px;"/>

Series don't have column indexes, because they don't have columns. You can, however, give your Series a name.

Note that DataFrame objects can also have a single-column – think of this as a DataFrame consisting of a single Series object (they might contain the same information, but are still distinct objects and are treated as such by `pandas`).

## Loading data into `pandas`

A popular way of working with data is storing it in **Comma-Separated Values (CSV) files** and then reading those files into Python using Pandas. CSVs are plain-text files that store tables using one CSV row per table row and separate the columns using commas (although other characters are possible, like semicolons):

<img src="data/images/csv.jpg" alt="A CSV file" style="width: 500px;"/>

To load a CSV file into `pandas`, you first import the library (the `pd` alias is customary) and then call the **`.read_csv()`** function like this:

~~~python
import pandas as pd
pd.read_csv("path/to/file.csv")
~~~

Let's read our dataset:

In [2]:
import pandas as pd
df = pd.read_csv("data/watches.csv")

We can inspect the first 5 rows using **`.head()`** (to see the last 5 we would use `.tail()`):

In [3]:
df.head()

,model,store,condition,engagement,price
0,Caracal,Watches unlimited,New,77.848101,489.0
1,Caracal,National traders,Like new,75.696203,489.0
2,Caracal,National traders,Good,72.025316,490.5
3,Lightning bolt,Super deals,Like new,78.987342,501.0
4,Sand,Super deals,Good,80.126582,502.5


This is a dataset of 75 watches that are sold online (some of them are new, but most are secondhand). There is data for three stores ("Watches unlimited", "National traders" and "Super deals"). The `engagement` column contains a metric of user engagement in the store website, and the `price` column is the price tag.

You can access a column using square brackets and quotes like this:

In [4]:
df["price"]

0      489.0
1      489.0
2      490.5
3      501.0
4      502.5
5      504.0
6      504.0
7      505.5
8      505.5
9      510.0
10     516.0
11     517.5
12     517.5
13     522.0
14     526.5
15     526.5
16     528.0
17     529.5
18     529.5
19     531.0
20     532.5
21     603.0
22     603.0
23     604.5
24     604.5
25     604.5
26     604.5
27     606.0
28     607.5
29     607.5
       ...  
45    4144.5
46    4144.5
47    4144.5
48    4147.5
49    4149.0
50    4149.0
51    4164.0
52    4165.5
53    4173.0
54    4176.0
55    4183.5
56    4185.0
57    4195.5
58    4195.5
59    4200.0
60    4203.0
61    4204.5
62    4206.0
63    4207.5
64    4210.5
65    4216.5
66    4216.5
67     831.0
68    4222.5
69    4233.0
70    4239.0
71    4282.5
72    4284.0
73     835.5
74    4308.0
Name: price, Length: 75, dtype: float64

By default, `pandas` assigns a serial number as the index (that's the unnamed column of integers in bold face in the `df.head()` output above), so setting a custom row index that makes sense for your DataFrame is often a good idea. You can set the index of a DataFrame with the **`.set_index()`** method:

In [5]:
df = df.set_index(["model", "store", "condition"])
df.head()

engagement  price
model          store             condition                   
Caracal        Watches unlimited New         77.848101  489.0
               National traders  Like new    75.696203  489.0
                                 Good        72.025316  490.5
Lightning bolt Super deals       Like new    78.987342  501.0
Sand           Super deals       Good        80.126582  502.5

Now the columns used as the indexes are emphasized in bold instead.

## Grouping your data

Grouping your data by a subset of the variables in the DataFrame is a common task in which `pandas` excels. We use the **`.groupby()`** method to do this. You can think of `groupby()` as a `for` loop that goes through the values of some user-specified column(s) and, in each iteration, extracts only those DataFrame rows that match that value in the specified column. The below diagram visualizes a `groupby()` operation over the column `foo`. Since there are only two values in `foo` (`one` and `two`), there are only two resulting groups:

<img src="data/images/groupby.jpg" alt="Groupby" style="width: 500px;"/>

Source: Modified from [`pandas` docs](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html).

When you do a group by, you're basically splitting your original DataFrame into smaller chunks, depending on the values of one or more columns.

The output of this method is a `DataFrameGroupBy` object. This object can be most readily thought of as containing a smaller DataFrame object for every group. Specifically, each item of the object is a tuple, containing the group identifier (in this case, `one` or `two`) and the rows of the DataFrame that correspond to that identifier.

`.groupby()` is most useful when we use it together with **aggregation functions** (functions that summarize our data), like `sum()`, `mean()`, and others. Here we group our dataframe `df` by the `model` and `condition` variables. Inside, the resulting object has all the data from `df`, partitioned into pieces depending on the values of the two variables we grouped by. We then ask `pandas` to compute the mean price for each category:

In [6]:
groups = df.groupby(["model", "condition"]) # Grouping by
groups["price"].mean() # You first slice using ["price"] and then call the mean function on the grouped Series

model           condition
Caracal         Fair         2949.5
                Good         1744.0
                Like new      508.0
                New          1818.5
                Very Good    1832.5
Clepsydra       Fair         4151.5
                Good          604.0
                Like new     1934.0
                New          4162.5
                Very Good    1737.0
Lightning bolt  Fair         4235.5
                Good          655.0
                Like new     1739.5
                New           652.5
                Very Good    1783.5
Sand            Fair         4207.5
                Good         3006.0
                Like new     1884.5
                New           543.5
                Very Good     520.0
Tempo           Fair         4189.5
                Good         4225.5
                Like new     3039.5
                New          2965.0
                Very Good    1854.5
Name: price, dtype: float64

If you'd like to see the contents of the `DataFrameGroupBy` object as such, you can iterate through it using a `for` loop like this:

~~~python
for group in groups:
    print(group)
~~~

### Exercise 1

Repeat the group by + aggregation above, only this time group by `model` alone instead of `model` and `condition`.

**Answer.**

In [8]:
groups1 = df.groupby("model")
df.head()

engagement  price
model          store             condition                   
Caracal        Watches unlimited New         77.848101  489.0
               National traders  Like new    75.696203  489.0
                                 Good        72.025316  490.5
Lightning bolt Super deals       Like new    78.987342  501.0
Sand           Super deals       Good        80.126582  502.5

In [9]:
for groups1 in groups:
    print(groups1)

(('Caracal', 'Fair'),                                      engagement   price
model   store             condition                    
Caracal Super deals       Fair        82.405063   505.5
        National traders  Fair        69.746835  4135.5
        Watches unlimited Fair        74.303797  4207.5)
(('Caracal', 'Good'),                                      engagement   price
model   store             condition                    
Caracal National traders  Good        72.025316   490.5
        Super deals       Good        81.139241   603.0
        Watches unlimited Good        72.784810  4138.5)
(('Caracal', 'Like new'),                                      engagement  price
model   store             condition                   
Caracal National traders  Like new    75.696203  489.0
        Super deals       Like new    76.202532  517.5
        Watches unlimited Like new    77.088608  517.5)
(('Caracal', 'New'),                                      engagement   price
model   store  

-------

## Pivoting

**Pivoting** is basically the same as grouping by + aggregating, with the difference that you can have some of the grouping variables as columns instead of as rows. This is sometimes nicer to look at.

Let's say you wanted to see the average `price` of each `model` per `condition`. Rather than doing the group by + aggregation we had above, we could use pivoting to get something like this (we made `condition` a set of columns this time):
<br><br>

<table border="1" class="dataframe">  <thead>    <tr>      <th></th>      <th colspan="5" halign="left">price</th>    </tr>    <tr>      <th>condition</th>      <th>Fair</th>      <th>Good</th>      <th>Like new</th>      <th>New</th>      <th>Very Good</th>    </tr>    <tr>      <th>model</th>      <th></th>      <th></th>      <th></th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Caracal</th>      <td>2949.5</td>      <td>1744.0</td>      <td>508.0</td>      <td>1818.5</td>      <td>1832.5</td>    </tr>    <tr>      <th>Clepsydra</th>      <td>4151.5</td>      <td>604.0</td>      <td>1934.0</td>      <td>4162.5</td>      <td>1737.0</td>    </tr>    <tr>      <th>Lightning bolt</th>      <td>4235.5</td>      <td>655.0</td>      <td>1739.5</td>      <td>652.5</td>      <td>1783.5</td>    </tr>    <tr>      <th>Sand</th>      <td>4207.5</td>      <td>3006.0</td>      <td>1884.5</td>      <td>543.5</td>      <td>520.0</td>    </tr>    <tr>      <th>Tempo</th>      <td>4189.5</td>      <td>4225.5</td>      <td>3039.5</td>      <td>2965.0</td>      <td>1854.5</td>    </tr>  </tbody></table>

This is called a **pivot table**.  You may already familiar with pivot tables if you have previously worked a fair amount in Microsoft Excel.

When you pivot, you summarize one of your numeric columns by group, where the groups are determined by some of the other columns. Some of those variables can be shown as rows, and some can be shown as columns. In the table above, we found the mean `price` grouping by `model` (shown as rows) and `condition` (shown as columns).

We create pivot tables in `pandas` using the **`.pivot_table()`** method, with this syntax:

~~~python
pd.pivot_table(my_df, values=["numeric_column"], index=["row_variable"], columns=["column_variable"])
~~~

`pandas` defaults to summarizing the numeric column using the `.mean()` function (commonly known as average or arithmetic mean). You can use other aggregation functions as well (for more information, check the [docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)).

### Exercise 2

Recreate the pivot table above using the syntax we've just explained and the variables from the `df` DataFrame.

**Answer.**

In [12]:
pd.pivot_table(df, values = ["price"], index = ["model"], columns=["condition"])

price                                   
condition         Fair    Good Like new     New Very Good
model                                                    
Caracal         2949.5  1744.0    508.0  1818.5    1832.5
Clepsydra       4151.5   604.0   1934.0  4162.5    1737.0
Lightning bolt  4235.5   655.0   1739.5   652.5    1783.5
Sand            4207.5  3006.0   1884.5   543.5     520.0
Tempo           4189.5  4225.5   3039.5  2965.0    1854.5

-------

## Unstacking

Let's have a look at our original dataset:

In [13]:
df

engagement   price
model          store             condition                    
Caracal        Watches unlimited New         77.848101   489.0
               National traders  Like new    75.696203   489.0
                                 Good        72.025316   490.5
Lightning bolt Super deals       Like new    78.987342   501.0
Sand           Super deals       Good        80.126582   502.5
               National traders  Very Good   79.493671   504.0
Lightning bolt Watches unlimited Very Good   78.860759   504.0
Clepsydra      National traders  Very Good   78.354430   505.5
Caracal        Super deals       Fair        82.405063   505.5
Sand           Watches unlimited New         79.493671   510.0
               National traders  New         78.734177   516.0
Caracal        Super deals       Like new    76.202532   517.5
               Watches unlimited Like new    77.088608   517.5
Lightning bolt Super deals       New         78.481013   522.0
Sand           Super deals       Very Good   79.367089   526.5
Lightning bolt Super deals       Good        80.126582   526.5
Caracal        Super deals       Very Good   80.759494   528.0
Clepsydra      Super deals       Very Good   77.215190   529.5
Sand           Watches unlimited Very Good   75.189873   529.5
Tempo          National traders  Very Good   76.455696   531.0
Lightning bolt National traders  Like new    79.113924   532.5
Caracal        Super deals       Good        81.139241   603.0
Clepsydra      Super deals       Good        81.012658   603.0
Lightning bolt Watches unlimited New         77.468354   604.5
Sand           Super deals       New         77.341772   604.5
Clepsydra      Watches unlimited Good        79.873418   604.5
               National traders  Good        80.759494   604.5
Tempo          Super deals       New         78.354430   606.0
Sand           National traders  Like new    75.063291   607.5
Lightning bolt Super deals       Very Good   79.240506   607.5
...                                                ...     ...
Clepsydra      Super deals       Fair        81.518987  4144.5
               National traders  Fair        83.164557  4144.5
               Super deals       New         78.481013  4144.5
               National traders  New         77.468354  4147.5
Tempo          Watches unlimited New         78.354430  4149.0
                                 Like new    77.088608  4149.0
               Super deals       Good        75.569620  4164.0
Clepsydra      Watches unlimited Fair        85.949367  4165.5
Tempo          Watches unlimited Fair        69.746835  4173.0
Clepsydra      Watches unlimited Very Good   80.632911  4176.0
Sand           National traders  Fair        83.291139  4183.5
Lightning bolt Watches unlimited Like new    74.430380  4185.0
Clepsydra      Watches unlimited New         77.974684  4195.5
Tempo          National traders  Fair        74.430380  4195.5
               Super deals       Fair        74.683544  4200.0
                                 Very Good   75.822785  4203.0
               National traders  Good        82.405063  4204.5
Lightning bolt Super deals       Fair        85.189873  4206.0
Caracal        Watches unlimited Fair        74.303797  4207.5
Sand           Watches unlimited Like new    78.354430  4210.5
                                 Fair        83.164557  4216.5
Lightning bolt National traders  Fair        86.202532  4216.5
               Watches unlimited Good        80.886076   831.0
Sand           Super deals       Fair        82.278481  4222.5
               Watches unlimited Good        81.012658  4233.0
Lightning bolt National traders  Very Good   80.379747  4239.0
Sand           National traders  Good        80.506329  4282.5
Lightning bolt Watches unlimited Fair        67.088608  4284.0
Sand           Super deals       Like new    79.493671   835.5
Tempo          Watches unlimited Good        82.405063  4308.0

[75 rows x 2 columns]

It has 75 rows and two columns. If we count the three indexes as columns, that makes 5 columns total.

We sometimes want to reduce the number of rows without removing any data points, like when we need to paste our table in a report and want it to fit on a single page. Additionally, having `condition` as a single column makes it difficult to locate the values that correspond to each possible condition ("New", "Like new", etc.). We can address both these issues using something called **unstacking**.

When you unstack a DataFrame, you take its innermost index level (in this case, `condition`) and turn it into a column of its own, and the DataFrame gets rearranged accordingly. This has the consequence of reducing the number of rows in your dataset while increasing the number of columns. In other words, when you unstack, you reshape your DataFrame to change it from a *long form* to a *wide form*.

Unstacking in `pandas` is very easy with the **`.unstack()`** method:

In [14]:
df.unstack("condition")

engagement                                   \
condition                              Fair       Good   Like new        New   
model          store                                                           
Caracal        National traders   69.746835  72.025316  75.696203  79.113924   
               Super deals        82.405063  81.139241  76.202532  79.620253   
               Watches unlimited  74.303797  72.784810  77.088608  77.848101   
Clepsydra      National traders   83.164557  80.759494  77.848101  77.468354   
               Super deals        81.518987  81.012658  77.468354  78.481013   
               Watches unlimited  85.949367  79.873418  79.620253  77.974684   
Lightning bolt National traders   86.202532  80.000000  79.113924  77.974684   
               Super deals        85.189873  80.126582  78.987342  78.481013   
               Watches unlimited  67.088608  80.886076  74.430380  77.468354   
Sand           National traders   83.291139  80.506329  75.063291  78.734177   
               Super deals        82.278481  80.126582  79.493671  77.341772   
               Watches unlimited  83.164557  81.012658  78.354430  79.493671   
Tempo          National traders   74.430380  82.405063  78.227848  77.974684   
               Super deals        74.683544  75.569620  78.101266  78.354430   
               Watches unlimited  69.746835  82.405063  77.088608  78.354430   

                                              price                           \
condition                         Very Good    Fair    Good Like new     New   
model          store                                                           
Caracal        National traders   79.240506  4135.5   490.5    489.0  4135.5   
               Super deals        80.759494   505.5   603.0    517.5   831.0   
               Watches unlimited  78.987342  4207.5  4138.5    517.5   489.0   
Clepsydra      National traders   78.354430  4144.5   604.5   4140.0  4147.5   
               Super deals        77.215190  4144.5   603.0    831.0  4144.5   
               Watches unlimited  80.632911  4165.5   604.5    831.0  4195.5   
Lightning bolt National traders   80.379747  4216.5   607.5    532.5   831.0   
               Super deals        79.240506  4206.0   526.5    501.0   522.0   
               Watches unlimited  78.860759  4284.0   831.0   4185.0   604.5   
Sand           National traders   79.493671  4183.5  4282.5    607.5   516.0   
               Super deals        79.367089  4222.5   502.5    835.5   604.5   
               Watches unlimited  75.189873  4216.5  4233.0   4210.5   510.0   
Tempo          National traders   76.455696  4195.5  4204.5    829.5  4140.0   
               Super deals        75.822785  4200.0  4164.0   4140.0   606.0   
               Watches unlimited  80.126582  4173.0  4308.0   4149.0  4149.0   

                                            
condition                        Very Good  
model          store                        
Caracal        National traders      831.0  
               Super deals           528.0  
               Watches unlimited    4138.5  
Clepsydra      National traders      505.5  
               Super deals           529.5  
               Watches unlimited    4176.0  
Lightning bolt National traders     4239.0  
               Super deals           607.5  
               Watches unlimited     504.0  
Sand           National traders      504.0  
               Super deals           526.5  
               Watches unlimited     529.5  
Tempo          National traders      531.0  
               Super deals          4203.0  
               Watches unlimited     829.5

Now we have 15 rows and 12 columns (before we had 75 rows and 5 columns). Since we also avoided having to repeat the variable names, this unstacked table takes a lot less space and can be read more easily.

### Exercise 3

What is the difference between pivoting and unstacking?

**Answer.**

In [ ]:
Pivoting gives you total control over the indices of your data, unstacking allows you to get to a subgroup and see it more detail

-------

### Stacking

**Stacking** is just the reverse of unstacking. You take a wide DataFrame and make it long. For instance, here we unstack `df`:

In [15]:
my_unstacked_df = df.unstack()
my_unstacked_df.head()

engagement                                   \
condition                         Fair       Good   Like new        New   
model     store                                                           
Caracal   National traders   69.746835  72.025316  75.696203  79.113924   
          Super deals        82.405063  81.139241  76.202532  79.620253   
          Watches unlimited  74.303797  72.784810  77.088608  77.848101   
Clepsydra National traders   83.164557  80.759494  77.848101  77.468354   
          Super deals        81.518987  81.012658  77.468354  78.481013   

                                         price                           \
condition                    Very Good    Fair    Good Like new     New   
model     store                                                           
Caracal   National traders   79.240506  4135.5   490.5    489.0  4135.5   
          Super deals        80.759494   505.5   603.0    517.5   831.0   
          Watches unlimited  78.987342  4207.5  4138.5    517.5   489.0   
Clepsydra National traders   78.354430  4144.5   604.5   4140.0  4147.5   
          Super deals        77.215190  4144.5   603.0    831.0  4144.5   

                                       
condition                   Very Good  
model     store                        
Caracal   National traders      831.0  
          Super deals           528.0  
          Watches unlimited    4138.5  
Clepsydra National traders      505.5  
          Super deals           529.5

We can reverse the process using **`.stack()`**.

### Exercise 4

Stack `my_unstacked_df` and verify that the result is exactly equal to `df` (the original DataFrame).

**Answer.**

In [18]:
my_stacked_df = my_unstacked_df.stack()
my_stacked_df.head ()
df.head()


engagement  price
model          store             condition                   
Caracal        Watches unlimited New         77.848101  489.0
               National traders  Like new    75.696203  489.0
                                 Good        72.025316  490.5
Lightning bolt Super deals       Like new    78.987342  501.0
Sand           Super deals       Good        80.126582  502.5

-------

---

If you are interested in learning more about DataFrame manipulation in `pandas`, we encourage you to study their very convenient [user guide](https://pandas.pydata.org/docs/user_guide/reshaping.html).

## Attribution

"Reshaping and pivot" (modified from the original), Pandas Developers, [BSD-3 license](https://github.com/pandas-dev/pandas/blob/master/LICENSE), https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html.